In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/My Drive/WPI/Junior/CS 525/homework/Homework3"

/content/drive/My Drive/WPI/Junior/CS 525/homework/Homework3


In [ ]:
!pip3 install transformers

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 38.2 MB/s 
     |████████████████████████████████| 59 kB 7.3 MB/s 
     |████████████████████████████████| 3.3 MB 37.8 MB/s 
     |████████████████████████████████| 596 kB 52.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch

In [ ]:
!nvidia-smi

Sat Nov 27 23:00:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [ ]:
# Inport squad dataset from train-v2.0.json
# Read json file
import json
with open('train-v2.0.json') as f:
	train_data = json.load(f)

In [ ]:
paragraphs = []
for article in train_data['data']:
	for para in article['paragraphs']:
		paragraphs.append(para['context'])

In [ ]:
print(paragraphs[0])

Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".


In [ ]:
# Get questions and answers
questions = []
answers = []
question_ids = []
question_map = {}
for article in train_data['data']:
    for para in article['paragraphs']:
        tmp_questions = []
        tmp_answers = []
        tmp_question_ids = []
        for qa in para['qas']:
            tmp_question_ids.append(qa['id'])
            try:
                question_map[qa['id']] = [qa['question'],
                                          para['context'], qa['answers'][0]['text']]
                tmp_questions.append(qa['question'])
                tmp_answers.append(qa['answers'][0]['text'])
            except:
                question_map[qa['id']] = [qa['question'], para['context']]
                print("No answers for: {}".format(qa['question']))
                tmp_questions.append("")
                tmp_answers.append("")
        questions.append(tmp_questions)
        answers.append(tmp_answers)
        question_ids.append(tmp_question_ids)


Streaming output truncated to the last 5000 lines.
No answers for: Who discovered the chemical structure of bacteria?
No answers for: What do polysaccharides need to work?
No answers for: What type of polysaccharides have saved millions of lives?
No answers for: What types of polysaccharides have been phased out?
No answers for: Vaccines made from what have been replaced by less protein carriers?
No answers for: What happens when a host includes polysaccharides?
No answers for: What has been talked about to treat bateriophages?
No answers for: How have bacterium been doing this?
No answers for: What is a worry of using DNA?
No answers for: What makes researchers confident that phages fight pathogens?
No answers for: By what method do phages destroy antibiotics?
No answers for: How many bacilli have been created in the last 7 years?
No answers for: How many are in the pipeline to fight IDSA?
No answers for: What year did the Infectious Diseases of America say that production of GNB does

In [ ]:
# Get question ids
# question_ids = []
# for article in train_data['data']:
# 	for para in article['paragraphs']:
# 		# tmp_question_ids = []
# 		for qa in para['qas']:
# 			question_ids.append(qa['id'])
		# question_ids.append(tmp_question_ids)

In [ ]:
print(len(question_map))

130319


In [ ]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model.to(device)


Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
question = "How many parameters does BERT-large have?"
answer_text = "BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take a couple minutes to download to your Colab instance."

In [ ]:
# Apply the tokenizer to the input text, treating them as a text-pair.
input_ids = tokenizer.encode(question, answer_text)

print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 70 tokens.


In [ ]:
# BERT only needs the token IDs, but for the purpose of inspecting the 
# tokenizer's behavior, let's also get the token strings and display them.
tokens = tokenizer.convert_ids_to_tokens(input_ids)

# For each token and its id...
for token, id in zip(tokens, input_ids):
    
    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')
    
    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')

[CLS]           101
how           2,129
many          2,116
parameters   11,709
does          2,515
bert         14,324
-             1,011
large         2,312
have          2,031
?             1,029

[SEP]           102

bert         14,324
-             1,011
large         2,312
is            2,003
really        2,428
big           2,502
.             1,012
.             1,012
.             1,012
it            2,009
has           2,038
24            2,484
-             1,011
layers        9,014
and           1,998
an            2,019
em            7,861
##bed         8,270
##ding        4,667
size          2,946
of            1,997
1             1,015
,             1,010
02            6,185
##4           2,549
,             1,010
for           2,005
a             1,037
total         2,561
of            1,997
340          16,029
##m           2,213
parameters   11,709
!               999
altogether   10,462
it            2,009
is            2,003
1             1,015
.             1,01

In [ ]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [ ]:
# Run our example through the model.
outputs = model(torch.tensor([input_ids]).to(device), # The tokens representing our input text.
                             token_type_ids=torch.tensor([segment_ids]).to(device), # The segment IDs to differentiate question from answer_text
                             return_dict=True) 

start_scores = outputs.start_logits
end_scores = outputs.end_logits


In [ ]:
# Find the tokens with the highest `start` and `end` scores.
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

# Combine the tokens in the answer and print it out.
answer = ' '.join(tokens[answer_start:answer_end+1])

print('Answer: "' + answer + '"')

Answer: "340 ##m"


In [ ]:
# Start with the first token.
answer = tokens[answer_start]

# Select the remaining answer tokens and join them with whitespace.
for i in range(answer_start + 1, answer_end + 1):
    
    # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
    
    # Otherwise, add a space then the token.
    else:
        answer += ' ' + tokens[i]

print('Answer: "' + answer + '"')

Answer: "340m"


In [ ]:
# Convert paragraph and question to tokens
tokenized_questions_map = {}
# def tokenize_paragraph(i):
for question in question_map:
	# print(question)
	tokenized_questions_map[question] = tokenizer.encode(question_map[question][0], question_map[question][1])


Token indices sequence length is longer than the specified maximum sequence length for this model (518 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
final_answers_map = {}
# Run model on each question
counter = 0
for question in tokenized_questions_map:
	counter += 1
	input_ids = tokenized_questions_map[question]

	tokens = tokenizer.convert_ids_to_tokens(input_ids)

	# Search the input_ids for the first instance of the `[SEP]` token.
	sep_index = input_ids.index(tokenizer.sep_token_id)

	# The number of segment A tokens includes the [SEP] token istelf.
	num_seg_a = sep_index + 1

	# The remainder are segment B.
	num_seg_b = len(input_ids) - num_seg_a

	# Construct the list of 0s and 1s.
	segment_ids = [0]*num_seg_a + [1]*num_seg_b

	# There should be a segment_id for every input token.
	assert len(segment_ids) == len(input_ids)
	try:
		outputs = model(torch.tensor([input_ids]).to(device), # The tokens representing our input text.
															token_type_ids=torch.tensor([segment_ids]).to(device), # The segment IDs to differentiate question from answer_text
															return_dict=True) 

		start_scores = outputs.start_logits
		end_scores = outputs.end_logits

		answer_start = torch.argmax(start_scores)
		answer_end = torch.argmax(end_scores)

		answer = tokens[answer_start]

		# Select the remaining answer tokens and join them with whitespace.
		for i in range(answer_start + 1, answer_end + 1):
			
			# If it's a subword token, then recombine it with the previous token.
			if tokens[i][0:2] == '##':
				answer += tokens[i][2:]
			
			# Otherwise, add a space then the token.
			else:
				answer += ' ' + tokens[i]

		print(str(counter) + ' Answer: "' + answer + '"')
		final_answers_map[question] = answer
	except:
		final_answers_map[question] = ""
	# if counter == 10:
		# break

# Write answers to file
with open('answers.json', 'w') as f:
	json.dump(final_answers_map, f)

Streaming output truncated to the last 5000 lines.
837Answer: "nicolas"
838Answer: "lorraine"
839Answer: "justyna krzyzanowska"
840Answer: "fryderyk skarbek"
841Answer: "ludwika"
842Answer: "nicolas"
843Answer: "lorraine"
844Answer: "ludwika"
845Answer: "october"
846Answer: "french"
847Answer: "the palace grounds"
848Answer: "flute and violin"
849Answer: "illnesses"
850Answer: "october 1810"
851Answer: "warsaw lyceum"
852Answer: "flute and violin"
853Answer: "piano"
854Answer: "six months"
855Answer: "french"
856Answer: "flute and violin"
857Answer: "piano"
858Answer: "saxon palace . fryderyk lived with his family in the palace grounds"
859Answer: "wojciech zywny"
860Answer: "ludwika"
861Answer: "seven"
862Answer: "two polonaises"
863Answer: "a polonaise in a - flat major"
864Answer: "wojciech zywny"
865Answer: "ludwika"
866Answer: "seven"
867Answer: "1817"
868Answer: "1816 to 1821"
869Answer: "wojciech zywny"
870Answer: "ludwika"
871Answer: "seven"
872Answer: "wojciech zywny"
873Answe

In [ ]:
# If the questions are not in final_answers_map set to empty string
for question in question_map:
	if question not in final_answers_map:
		final_answers_map[question] = ""
		
with open('answers.json', 'w') as f:
	json.dump(final_answers_map, f)